# Deep Learning for Instance Segmentation of Agricultural Fields

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import itertools
import numpy as np
import random
from typing import List, Dict, Tuple

import rasterio
import rasterio.plot
import geopandas as gpd
import shapely
from shapely.geometry import Polygon
from PIL import Image as pilimg
from skimage import exposure, img_as_ubyte
from tqdm import tqdm
import matplotlib.pyplot as plt
import cgeo
import utils
import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocessing

The preprocessing workflow brings the image and geometry data to the COCO format expected by the FCIS model. It cuts the Sentinel-2 images to many small RGB image chips. The LPIS field parcel vectors are clipped to matching chip geometries and saved in COCO json format.   

Before the chipping tasks, a geopandas pipeline cleans up lass labels and geometries and clips them to the region of interest defined by the Sentinel-2 scene tiles.

As imagery and geometry data for the full Denmark roi of the thesis take up several gigabytes in file size, the repository just contains a small subset region for demonstration purposes. The full 2016 LPIS "marker" dataset set can be downloaded here:
https://kortdata.fvm.dk/download/Markblokke_Marker?page=MarkerHistoriske

![Subset roi](msc_codeshare/test_aoi_subset.png)

In [8]:
fp_s2 = Path(r'data\original\RGB_small_cor.tif')
fp_fields = Path(r'data\original\marker_small.shp')

In [17]:
with rasterio.open(fp_s2) as src:
    meta = src.meta
    bounds = src.bounds

def preprocess_vector(inpath, meta, bounds):
    df = (gpd.read_file(str(inpath), encoding='utf-8')  # contains danish special characters
             .rename(columns={'Afgroede': 'lc_name', 'AfgKode': 'lc_id', 'JOURNALNUM': 'journalnr'})
             .drop(['GB', 'GEOMETRISK', 'MARKNUMMER'], axis=1)
             .pipe(cgeo.geo.buffer_zero)
             .pipe(cgeo.geo.close_holes)
             .pipe(cgeo.geo.set_crs, 3044)
             .to_crs(meta['crs'])
             .pipe(cgeo.geo.clip, clip_poly=shapely.geometry.box(*bounds), explode_mp_=True)
             .pipe(cgeo.geo.reclassify_col, rcl_scheme=preproc.reclass_legend, col_classlabels='lc_name',
                   col_classids='lc_id', drop_other_classes=True)
             .assign(geometry=lambda _df: _df.geometry.simplify(5, preserve_topology=True))
             .pipe(cgeo.geo.buffer_zero)
             .assign(area_sqm=lambda _df: _df.geometry.area)
             .pipe(cgeo.geo.reduce_precision, precision=4)
             .reset_index(drop=True)
             .assign(fid=lambda _df: range(0, len(_df.index)))
             .filter(['journalnr', 'lc_id', 'lc_name', 'rcl_lc_id', 'rcl_lc_name', 'area_sqm', 'fid', 'geometry'])
          )
    return df


df = cgeo.other.read_or_new_save(path=Path(r'output\preprocessed\preprocessed_geo.pkl'),
                                 default_data=preprocess_vector,
                                 callable_args={'inpath': fp_fields, 'meta': meta, 'bounds': bounds})

print('df.info()', df.info())
display(df.head(3))
# print('overall number of fields\n', len(df))
# print('\nnumber of fields per class\n', df.groupby(['rcl_lcsub']).fid.aggregate(len).sort_values(ascending=False))
# print('\noverall mean area of fields\n', df.area_sqm.mean())
# print('\nmean area of fields per class\n', df.groupby(['rcl_lcsub']).area_sqm.mean().sort_values(ascending=False))

Writing new pickle file... preprocessed_geo.pkl
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 8 columns):
journalnr      113 non-null object
lc_id          113 non-null int64
lc_name        113 non-null object
rcl_lc_id      113 non-null int64
rcl_lc_name    113 non-null object
area_sqm       113 non-null float64
fid            113 non-null int32
geometry       113 non-null object
dtypes: float64(1), int32(1), int64(2), object(4)
memory usage: 6.7+ KB
df.info() None


,journalnr,lc_id,lc_name,rcl_lc_id,rcl_lc_name,area_sqm,fid,geometry
0,16-0036135,30,Ærter,5,other,18409.058038,0,"POLYGON ((513952.7684 6243596.0941, 514026.304..."
1,16-0044708,1,Vårbyg,1,springcereal,343798.036407,1,"POLYGON ((515394.05 6247182.35, 515511.8944 62..."
2,16-0053716,260,Græs med kløver/lucerne,4,grassfields,202990.290354,2,"POLYGON ((512418.2498 6244766.2459, 512430.503..."


In [ ]:
# # Cut chips for images and geometries

def cut_chips(img_path, df, chip_width, chip_height, bands):
    chips_stats = {}
    src = rasterio.open(img_path)
    generator_window_bounds = cgeo.img.get_chip_windows(meta_raster=src.meta,
                                                        chip_width=chip_width,
                                                        chip_height=chip_height,
                                                        skip_partial_chips=True)

    for i, (chip_window, chip_poly, chip_transform) in enumerate(tqdm(generator_window_bounds)):
        if i % 100 == 0: print(i)

        # # Clip geometry to chip
        chip_df = df.pipe(cgeo.geo.clip, clip_poly=chip_poly, keep_biggest_poly_=True)
        if not all(chip_df.geometry.is_empty):
            chip_df.geometry = chip_df.simplify(1, preserve_topology=True)
        else:
            continue
        chip_df = chip_df[chip_df.geometry.area * (10 * 10) > 5000]
        # Transform to chip pixelcoordinates and invert y-axis for COCO format.
        if not all(chip_df.geometry.is_empty):
            chip_df = chip_df.pipe(cgeo.geo.to_pixelcoords, reference_bounds=chip_poly.bounds, scale=True,
                                   ncols=chip_width, nrows=chip_height)
            chip_df = chip_df.pipe(cgeo.geo.invert_y_axis, reference_height=chip_height)
        else:
            continue

        # # Clip image to chip
        img_array = np.dstack(list(src.read(bands, window=chip_window)))
        img_array = exposure.rescale_intensity(img_array, in_range=(0, 2200))  # Sentinel2 range.
        img_array = img_as_ubyte(img_array)
        img_pil = pilimg.fromarray(img_array)

        # # Export image chip.
        for folder in ['train2014', 'val2014']:
            Path(rf'output\preprocessed\chips\{folder}').mkdir(parents=True, exist_ok=True)
        chip_file_name = f'COCO_train2014_000000{100000+i}'  # _{clip_minX}_{clip_minY}_{clip_maxX}_{clip_maxY}'
        with open(Path(rf'output\preprocessed\\chips\train2014\{chip_file_name}.jpg'), 'w') as dst:
            img_pil.save(dst, format='JPEG', subsampling=0, quality=100)

        # # Gather image statistics
        chips_stats[chip_file_name] = {'chip_df': chip_df,
                                       'mean': img_array.mean(axis=(0, 1)),
                                       'std': img_array.std(axis=(0, 1))}
    src.close()
    return chips_stats


chip_width, chip_height = 128, 128
bands = [3, 2, 1]
chips_stats = cgeo.other.read_or_new_save(path=Path(r'output\preprocessed\chips_geo_stats.pkl'),
                                          default_data=cut_chips,
                                          callable_args={'img_path': fp_s2,
                                                         'df': df,
                                                         'chip_width': chip_width,
                                                         'chip_height': chip_height,
                                                         'bands': bands})

print('len(chips_stats)', len(chips_stats))

In [ ]:
def train_test_split_coco(chips_stats: Dict) -> Tuple[List, List]:
    chips_list = list(chips_stats.keys())
    random.seed(1)
    random.shuffle(chips_list)
    split_idx = round(len(chips_list) * 0.2)  # 80% train, 20% test.
    chips_train = chips_list[split_idx:]
    chips_val = chips_list[:split_idx]

    return chips_train, chips_val


def apply_split_coco(chips_train: List, chips_val: List) -> Tuple[Dict, Dict]:
    # Apply split to geometries/stats.
    stats_train = {k: chips_stats[k] for k in sorted(chips_train)}
    stats_val = {k.replace('train', 'val'): chips_stats[k] for k in sorted(chips_val)}
    print(len(chips_train), len(chips_val))

    # Apply split to image chips: Move val chip images.
    for chip in chips_val:
        destination = Path(r"output\preprocessed\chips\val2014\{}.jpg".format(chip.replace('train', 'val')))
        Path(rf"output\preprocessed\chips\train2014\{chip}.jpg").replace(destination)

    return stats_train, stats_val


def format_cocojson(set_: Dict):
    """
    Format extracted chip geometries to COCO json format.

    Coco train/val have specific ids, formatting requires the split data..
    """
    cocojson = {
        "info": {},
        "licenses": [],
        'categories': [{'supercategory': 'AgriculturalFields', 'id': 1, 'name': 'agfields_singleclass'}]}
    # id needs to match category_id.

    for key_idx, key in enumerate(set_.keys()):
        if 'train' in key:
            chip_id = int(key[21:])
        elif 'val' in key:
            chip_id = int(key[19:])

        key_image = ({"file_name": f'{key}.jpg',
                      "id": int(chip_id),
                      "height": chip_width,
                      "width": chip_height})
        cocojson.setdefault('images', []).append(key_image)

        for row_idx, row in set_[key]['chip_df'].iterrows():
            # Convert poly to COCO segmentation format, from shapely POLYGON ((x y, x1 y2, ..)) to COCO [[x, y, x1, y1,
            # ..]]. Except for crowd region (iscrowd=1), the annotations were encoded by RLE.
            coco_xy = list(itertools.chain.from_iterable((x, y) for x, y in zip(*row.geometry.exterior.coords.xy)))
            coco_xy = [round(xy, 2) for xy in coco_xy]
            # Add bbox.
            bounds = row.geometry.bounds  # COCO bbox format [minx, miny, width, height]
            coco_bbox = [bounds[0], bounds[1], bounds[2] - bounds[0], bounds[3] - bounds[1]]
            coco_bbox = [round(xy, 2) for xy in coco_bbox]

            key_annotation = {"id": key_idx,
                              "image_id": int(chip_id),
                              "category_id": 1,  # with multiclass "category_id" : row.reclass_lcsub_id
                              "mycategory_name": 'agfields_singleclass',
                              "old_multiclass_category_name": row['rcl_lcsub'],
                              "old_multiclass_category_id": row['rcl_lcsub_id'],
                              "bbox": coco_bbox,
                              "area": row.geometry.area,
                              "iscrowd": 0,
                              "segmentation": [coco_xy]}
            cocojson.setdefault('annotations', []).append(key_annotation)

    return cocojson


outpath_cocojson_train = Path(r'output\preprocessed\chips\train2014.json')
outpath_cocojson_val = Path(r'output\preprocessed\chips\val2014.json')

if outpath_cocojson_train.exists() and outpath_cocojson_val.exists():
    cocojson_train = cgeo.other.read_saved(outpath_cocojson_train, file_format='json')
    cocojson_val = cgeo.other.read_saved(outpath_cocojson_val, file_format='json')
else:
    chips_train, chips_val = train_test_split_coco(chips_stats)
    stats_train, stats_val = apply_split_coco(chips_train, chips_val)
    cocojson_train = format_cocojson(stats_train)
    cocojson_val = format_cocojson(stats_val)
    cgeo.other.new_save(outpath_cocojson_train, cocojson_train, file_format='json')
    cgeo.other.new_save(outpath_cocojson_val, cocojson_val, file_format='json')

In [ ]:
# # Gather chip statistics.
def get_statistics():
    return {
        'nr_chips': len(chips_stats.keys()),
        'nr_chips_train': len(chips_train),
        'nr_chips_val': len(chips_val),

        'nr_polys': sum([len(df['chip_df']) for df in chips_stats.values()]),
        'avg_polys_per_chip': sum([len(df['chip_df']) for df in chips_stats.values()]) / len(chips_stats.keys()),
        'nr_polys_train': sum([len(df['chip_df']) for df in [chips_stats[key] for key in chips_train]]),
        'nr_polys_val': sum([len(df['chip_df']) for df in [chips_stats[key] for key in chips_val]]),

        'train_rgb_mean': list(np.asarray([df['mean'] for df in [chips_stats[key] for key in chips_train]]).mean(axis=0)),
        'train_rgb_stdn': list(np.asarray([df['std'] for df in [chips_stats[key] for key in chips_train]]).mean(axis=0)),
        # 'polys_classstats': afterchip_df.groupby(['reclass_lcsub']).object_id.aggregate(len).sort_values(ascending=False),
        # 'polys_mean_sqm': afterchip_df.areasqm.mean(),
        # 'polys_classareastats:': afterchip_df.groupby(['reclass_lcsub']).areasqm.mean(),
    }

statistics = cgeo.other.read_or_new_save(path=Path(r'output\preprocessed\statistics.json'),
                                         default_data=get_statistics,
                                         file_format='json')
pprint.pprint(statistics)